In [8]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import matplotlib.pyplot as plot
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime

In [28]:
file = "./demographic_PA/census_block_demographic_PA.geojson"  
block_pop = gp.read_file(file)
block_pop.head()

,STATEFP10,GEOID10,GEOID,B02001e1,B02001e2,B02001e3,B02001e4,B02001e5,B02001e6,B03002e12,geometry
0,42,420035003001,15000US420035003001,1518,1509,0,0,0,0,0,"MULTIPOLYGON (((-79.86995 40.34914, -79.86980 ..."
1,42,420034994002,15000US420034994002,911,911,0,0,0,0,49,"MULTIPOLYGON (((-79.88670 40.32117, -79.88671 ..."
2,42,420034994001,15000US420034994001,775,756,19,0,0,0,19,"MULTIPOLYGON (((-79.88564 40.31210, -79.88689 ..."
3,42,420034993002,15000US420034993002,965,965,0,0,0,0,0,"MULTIPOLYGON (((-79.88271 40.33222, -79.88121 ..."
4,42,420034980002,15000US420034980002,1368,1350,13,0,0,0,15,"MULTIPOLYGON (((-79.86238 40.31546, -79.86247 ..."


In [10]:
#Rename columns
block_pop.rename(columns = {"B02001e6":"hawaiian"}, inplace = True)
block_pop.rename(columns = {"B03002e12":"hispanic_latino"}, inplace = True)
block_pop.rename(columns = {"B02001e5":"asian"}, inplace = True)
block_pop.rename(columns = {"B02001e2":"white"}, inplace = True)
block_pop.rename(columns = {"B02001e3":"black"}, inplace = True)
block_pop.rename(columns = {"B02001e1":"total"}, inplace = True)
block_pop.rename(columns = {"B02001e4":"indian_alaska"}, inplace = True)

#Drop column
#df = df.drop (columns = ['STATEFP10', 'GEOID'])
block_pop.head()

,STATEFP10,GEOID10,GEOID,total,white,black,indian_alaska,asian,hawaiian,hispanic_latino,geometry
0,42,420035003001,15000US420035003001,1518,1509,0,0,0,0,0,"MULTIPOLYGON (((-79.86995 40.34914, -79.86980 ..."
1,42,420034994002,15000US420034994002,911,911,0,0,0,0,49,"MULTIPOLYGON (((-79.88670 40.32117, -79.88671 ..."
2,42,420034994001,15000US420034994001,775,756,19,0,0,0,19,"MULTIPOLYGON (((-79.88564 40.31210, -79.88689 ..."
3,42,420034993002,15000US420034993002,965,965,0,0,0,0,0,"MULTIPOLYGON (((-79.88271 40.33222, -79.88121 ..."
4,42,420034980002,15000US420034980002,1368,1350,13,0,0,0,15,"MULTIPOLYGON (((-79.86238 40.31546, -79.86247 ..."


In [11]:
file = "./PA/PA_final.geojson"  
precincts = gp.read_file(file)
precincts.head()

,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.157814,-80.479783,0.004278359631999892,"4207300790, 4208500950, 4208501040, 4208500425...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,ORIGINAL,41.680613,-80.04085000000001,0.0005506003439999464,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."
2,ORIGINAL,41.743054,-79.840985,0.007933951458499875,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.83310 41.775..."
3,ORIGINAL,41.803153,-80.462641,0.01028330178850014,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.40589 41.849..."
4,ORIGINAL,41.812444,-79.836173,0.01073166413550007,"4203900010, 4203900400, 4203900350, 4204901060...",PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.76203 41.850..."


In [14]:
precinct_demographics = precincts[['id', 'geometry']].copy()

#Add type columns
precinct_demographics.insert(1, 'total', 0)
precinct_demographics.insert(2, 'white', 0)
precinct_demographics.insert(3, 'black', 0)
precinct_demographics.insert(4, 'hispanic_latino', 0)
precinct_demographics.insert(5, 'asian', 0)
precinct_demographics.insert(6, 'indian_alaska', 0)
precinct_demographics.insert(7, 'hawaiian', 0)

precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,4208500960,0,0,0,0,0,0,0,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,4203900040,0,0,0,0,0,0,0,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."
2,4203900010,0,0,0,0,0,0,0,"POLYGON ((-79.90979 41.77394, -79.83310 41.775..."
3,4203900020,0,0,0,0,0,0,0,"POLYGON ((-80.51935 41.84956, -80.40589 41.849..."
4,4203900030,0,0,0,0,0,0,0,"POLYGON ((-79.90998 41.85069, -79.76203 41.850..."


In [15]:
#Data assign based if census block or precinct enclosed
for i_precinct, row_precinct in precinct_demographics.iterrows():
    if i_precinct % 1000 == 0:
            print (datetime.datetime.now().strftime("%H:%M:%S"), i_precinct)    
    for i_block, row_block in block_pop.iterrows():
        
        if row_precinct.geometry.contains(row_block.geometry):
            print ("Precinct", row_precinct['id'], 'contains block ', i_block)
            precinct_demographics.at[i_precinct, 'total'] += row_block['total']
            precinct_demographics.at[i_precinct, 'white'] += row_block['white']
            precinct_demographics.at[i_precinct, 'black'] += row_block['black']
            precinct_demographics.at[i_precinct, 'hispanic_latino'] += row_block['hispanic_latino']
            precinct_demographics.at[i_precinct, 'asian'] += row_block['asian']
            precinct_demographics.at[i_precinct, 'indian_alaska'] += row_block['indian_alaska']
            precinct_demographics.at[i_precinct, 'hawaiian'] += row_block['hawaiian']
            block_pop = block_pop.drop(i_block)
    

09:51:17 0
10:06:32 1000
Precinct 4207500260 contains block  4370
Precinct 4213301240 contains block  2466
Precinct 4213300880 contains block  1982
Precinct 4213300890 contains block  2855
Precinct 4207901775 contains block  5987
Precinct 4213300900 contains block  2013
Precinct 4213300900 contains block  2015
Precinct 4213301200 contains block  2371
Precinct 4209700540 contains block  6538
Precinct 4211900060 contains block  4911
10:22:05 2000
Precinct 4212300060 contains block  5813
Precinct 4212300231 contains block  5817
Precinct 4209900040 contains block  2771
Precinct 4209103280 contains block  6702
Precinct 4209102610 contains block  6103
Precinct 4209102620 contains block  6315
Precinct 4209102160 contains block  6087
10:37:19 3000
Precinct 4210701440 contains block  4145
Precinct 4204300420 contains block  4236
Precinct 4207700380 contains block  5354
Precinct 4207701130 contains block  5780
Precinct 4207700090 contains block  5781
Precinct 4211100300 contains block  7054
10:5

In [16]:
#Data assign based on how much census block area inside of a precinct 
for i_precinct, row_precinct in precincts.iterrows():
    if i_precinct % 1000 == 0:
            print (datetime.datetime.now().strftime("%H:%M:%S"), i_precinct)
            
    for i_block, row_block in block_pop.iterrows():
        intersection = row_precinct.geometry.intersection(row_block.geometry)
        if intersection.area > 0:
            ratio = intersection.area / row_block.geometry.area
            if ratio > 1:
                print (i_precinct)
            precinct_demographics.at[i_precinct, 'total'] += round(row_block['total'] * ratio)
            precinct_demographics.at[i_precinct, 'white'] += round(row_block['white'] * ratio)
            precinct_demographics.at[i_precinct, 'black'] += round(row_block['black'] * ratio)
            precinct_demographics.at[i_precinct, 'hispanic_latino'] += round(row_block['hispanic_latino'] * ratio)
            precinct_demographics.at[i_precinct, 'asian'] += round(row_block['asian'] * ratio)
            precinct_demographics.at[i_precinct, 'indian_alaska'] += round(row_block['indian_alaska'] * ratio)
            precinct_demographics.at[i_precinct, 'hawaiian'] += round(row_block['hawaiian'] * ratio)

12:11:57 0
12:32:45 1000
12:54:00 2000
13:15:24 3000
13:36:09 4000
13:57:05 5000
14:17:53 6000
14:38:45 7000
14:59:31 8000
15:20:22 9000


In [19]:
precinct_demographics = precinct_demographics.drop(columns = ['geometry'])
pd.DataFrame(precinct_demographics).to_json("./demographic_PA/precinct_demographic_PA.json")

In [17]:
precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,4208500960,1700,1670,0,0,10,0,0,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,4203900040,158,154,0,0,3,0,0,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."
2,4203900010,801,798,0,16,0,0,0,"POLYGON ((-79.90979 41.77394, -79.83310 41.775..."
3,4203900020,771,766,2,0,0,0,0,"POLYGON ((-80.51935 41.84956, -80.40589 41.849..."
4,4203900030,2092,2044,9,26,0,5,0,"POLYGON ((-79.90998 41.85069, -79.76203 41.850..."


In [58]:
precinct_demographics = pd.read_json('./demographic_PA/precinct_demographic_PA.json')
precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
0,4208500960,1700,1670,0,0,10,0,0
1,4203900040,158,154,0,0,3,0,0
2,4203900010,801,798,0,16,0,0,0
3,4203900020,771,766,2,0,0,0,0
4,4203900030,2092,2044,9,26,0,5,0


In [65]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [66]:
add_demographic_querry = ("INSERT INTO mhaczynska.DEMOGRAPHIC_DATA "
               "(PRECINCT_GEOID, TOTAL_POPULATION, WHITE, "
               " BLACK, HISPANIC_LATINO, ASIAN, INDIAN_ALASKAN, HAWAIIAN_PACIFIC) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")

update_precinct_querry = ("UPDATE mhaczynska.DEMOGRAPHIC_DATA SET TOTAL_POPULATION = %s, WHITE = %s, BLACK = %s, "
" HISPANIC_LATINO = %s, ASIAN = %s, INDIAN_ALASKAN = %s, HAWAIIAN_PACIFIC = %s WHERE PRECINCT_GEOID = %s")
STATE = 'PA'
for index, row in precinct_demographics.iterrows(): 
    #demographic_data = (row['id'], row['total'], row['white'], row['black'] , 
                       # row['hispanic_latino'], row['asian'], row['indian_alaska'], row['hawaiian'])
    update_precinct = (row['total'], row['white'], row['black'], row['hispanic_latino'], row['asian'], row['indian_alaska'], 
                       row['hawaiian'], row['id'])
    cursor.execute(update_precinct_querry, update_precinct)    
    mydb.commit()
    #print(update_precinct)
    
    

In [53]:
### ANOMALOUS DATA
file = "./demographic_PA/precinct_demographic_PA.json"
demographics = pd.read_json(file)
demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
0,4208500960,1700,1670,0,0,10,0,0
1,4203900040,158,154,0,0,3,0,0
2,4203900010,801,798,0,16,0,0,0
3,4203900020,771,766,2,0,0,0,0
4,4203900030,2092,2044,9,26,0,5,0


In [27]:
demographics[demographics['id'] == '42049ZZZZZ'].head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
6595,42049ZZZZZ,7498,6883,411,133,62,5,0


In [33]:
empty_df = block_pop[block_pop['B02001e1'] == 0]

In [48]:
empty_df.shape

(22, 11)

In [38]:
file = "./PA/PA_final.geojson"  
precincts = gp.read_file(file)
precincts.head()

,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.157814,-80.479783,0.004278359631999892,"4207300790, 4208500950, 4208501040, 4208500425...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,ORIGINAL,41.680613,-80.04085000000001,0.0005506003439999464,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."
2,ORIGINAL,41.743054,-79.840985,0.007933951458499875,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.83310 41.775..."
3,ORIGINAL,41.803153,-80.462641,0.01028330178850014,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.40589 41.849..."
4,ORIGINAL,41.812444,-79.836173,0.01073166413550007,"4203900010, 4203900400, 4203900350, 4204901060...",PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.76203 41.850..."


In [ ]:
for i, row in precincts.iterrows():
    for j, block in empty_df.iterrows():
        intersection = row.geometry.intersection(block.geometry)
        if intersection.area > 0:
            ratio = intersection.area / block.geometry.area
            if ratio > 0.90:
                print(ratio, row['id'], block['GEOID10'])
                #demographics.at[i, 'total'] = 0
                #demographics.at[i, 'white'] = 0
                #demographics.at[i, 'hispanic_latino'] = 0
                #demographics.at[i, 'asian'] = 0
                #demographics.at[i, 'indian_alaska'] = 0
                #demographics.at[i, 'hawaiian'] = 0

0.9922947118439299 4201701050 420179800001
0.9938120599499495 4207902495 420799801001
0.9943837761459577 4204502960 420459800001


In [56]:
demographics[demographics['total'] == 0].shape

(34, 8)